In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import zipfile

# Path to the ZIP file
zip_path = "/content/drive/MyDrive/dataset_crossvit-20250331T231403Z-001.zip"

# Path to extract the dataset
extract_to = "/content/drive/MyDrive/dataset_crossvit"

# Function to extract the ZIP file
def extract_zip(zip_path, extract_to):
    if not os.path.exists(extract_to):
        os.makedirs(extract_to)
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print(f"Extracted ZIP file to {extract_to}")

# Execute extraction
extract_zip(zip_path, extract_to)


Extracted ZIP file to /content/drive/MyDrive/dataset_crossvit


In [ ]:
print(os.listdir(extract_to))


['dataset_crossvit']


### Overfitting but good accuracy on train but no generalisation

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),  # Slight color variation
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    # Add small Gaussian noise
    transforms.Lambda(lambda x: x + torch.randn_like(x) * 0.01)  # Small noise to simulate chart variations
])

In [ ]:
valid_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, weight=None, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.weight = weight
        self.reduction = reduction

    def forward(self, input, target):
        ce_loss = F.cross_entropy(input, target, weight=self.weight, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = (1 - pt) ** self.gamma * ce_loss
        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        return focal_loss

class RoboflowCocoDataset(Dataset):
    def __init__(self, root_dir, annotation_file, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        with open(annotation_file, 'r') as f:
            self.coco_data = json.load(f)
        self.category_map = {cat['id']: idx for idx, cat in enumerate(self.coco_data['categories'])}
        self.class_names = [cat['name'] for cat in self.coco_data['categories']]
        self.image_info = {img['id']: img['file_name'] for img in self.coco_data['images']}
        self.annotations = {}
        for ann in self.coco_data['annotations']:
            img_id = ann['image_id']
            if img_id not in self.annotations:
                self.annotations[img_id] = []
            self.annotations[img_id].append(ann['category_id'])
        self.valid_image_ids = [img_id for img_id in self.image_info.keys() if img_id in self.annotations]
        print(f"Total images: {len(self.image_info)}, Images with annotations: {len(self.valid_image_ids)}")
        missing = set(self.image_info.keys()) - set(self.annotations.keys())
        if missing:
            print(f"Images missing annotations: {len(missing)}. Example IDs: {list(missing)[:5]}")

        # Class distribution
        labels = [self.annotations[img_id][0] for img_id in self.valid_image_ids]
        self.class_dist = Counter([self.class_names[self.category_map[l]] for l in labels])
        print("Class distribution:", self.class_dist)
        self.labels = [self.category_map[self.annotations[img_id][0]] for img_id in self.valid_image_ids]

    def __len__(self):
        return len(self.valid_image_ids)

    def __getitem__(self, idx):
        img_id = self.valid_image_ids[idx]
        img_path = os.path.join(self.root_dir, self.image_info[img_id])
        image = Image.open(img_path).convert('RGB')
        category_id = self.annotations[img_id][0]
        label = self.category_map[category_id]
        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:
import os
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import timm
from torch.optim import Adam
from PIL import Image
from collections import Counter
import numpy as np

# Load datasets
train_annotation_file = os.path.join(train_dir, '_annotations.coco.json')
valid_annotation_file = os.path.join(valid_dir, '_annotations.coco.json')

train_dataset = RoboflowCocoDataset(train_dir, train_annotation_file, transform=train_transform)
valid_dataset = RoboflowCocoDataset(valid_dir, valid_annotation_file, transform=valid_transform)

# Weighted sampling for training
class_counts = [train_dataset.class_dist[name] for name in train_dataset.class_names]
weights = 1.0 / torch.tensor(class_counts, dtype=torch.float)
sample_weights = torch.tensor([weights[label] for label in train_dataset.labels], dtype=torch.float)
sampler = WeightedRandomSampler(sample_weights, len(sample_weights))

train_loader = DataLoader(train_dataset, batch_size=32, sampler=sampler, num_workers=2)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False, num_workers=2)

print("Classes:", train_dataset.class_names)
print(f"Training samples: {len(train_dataset)}, Validation samples: {len(valid_dataset)}")
num_classes = len(train_dataset.class_names)

# Compute class weights for Focal Loss
total_samples = sum(class_counts)
weights = [total_samples / (num_classes * count) for count in class_counts]
weights = torch.tensor(weights, dtype=torch.float32)

model = timm.create_model('crossvit_15_240', pretrained=True, num_classes=num_classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print("Model loaded on:", device)

Total images: 1176, Images with annotations: 1170
Images missing annotations: 6. Example IDs: [73, 747, 785, 537, 1019]
Class distribution: Counter({'Triple Bottom Reversal': 332, 'Ascending Triangle': 211, 'Symmetric Triangle': 177, 'Descending Triangle': 114, 'Head and Shoulder': 110, 'Double Top': 84, 'Inverse Head and Shoulder': 55, 'Cup and Handle': 40, 'Double Bottom': 23, 'flag-and-pole': 22, 'Falling Wedge': 2})
Total images: 289, Images with annotations: 288
Images missing annotations: 1. Example IDs: [52]
Class distribution: Counter({'Triple Bottom Reversal': 77, 'Ascending Triangle': 68, 'Symmetric Triangle': 35, 'Descending Triangle': 31, 'Double Top': 23, 'Head and Shoulder': 17, 'Inverse Head and Shoulder': 14, 'Double Bottom': 12, 'Cup and Handle': 6, 'flag-and-pole': 5})
Classes: ['flag-and-pole', 'Ascending Triangle', 'Cup and Handle', 'Descending Triangle', 'Double Bottom', 'Double Top', 'Falling Wedge', 'Head and Shoulder', 'Inverse Head and Shoulder', 'Symmetric Tri

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/110M [00:00<?, ?B/s]

Model loaded on: cuda


In [ ]:
def train_and_evaluate(model, train_loader, valid_loader, num_epochs=100, lr=0.0001):
    optimizer = Adam(model.parameters(), lr=lr, weight_decay=1e-4)
    criterion = FocalLoss(gamma=2.0, weight=weights.to(device)).to(device)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5)

    # Freeze backbone initially
    for param in model.parameters():
        param.requires_grad = False
    for param in model.head.parameters():
        param.requires_grad = True

    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}

    for epoch in range(num_epochs):
        if epoch == 10:
            for param in model.parameters():
                param.requires_grad = True
            print("Unfreezing all layers")

        model.train()
        train_loss, correct_train, total_train = 0.0, 0, 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            train_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct_train += (predicted == labels).sum().item()
            total_train += labels.size(0)
        train_loss /= len(train_loader.dataset)
        train_acc = correct_train / total_train

        model.eval()
        val_loss, correct_val, total_val = 0.0, 0, 0
        with torch.no_grad():
            for inputs, labels in valid_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                correct_val += (predicted == labels).sum().item()
                total_val += labels.size(0)
        val_loss /= len(valid_loader.dataset)
        val_acc = correct_val / total_val

        scheduler.step(val_loss)

        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)

        print(f"Epoch {epoch+1}/{num_epochs}, "
              f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, "
              f"LR: {optimizer.param_groups[0]['lr']:.6f}")

    return history

In [ ]:
# Train and Evaluate
history = train_and_evaluate(model, train_loader, valid_loader, num_epochs=100)

Epoch 1/100, Train Loss: 11.3297, Train Acc: 0.0718, Val Loss: 1.8051, Val Acc: 0.0000, LR: 0.000100
Epoch 2/100, Train Loss: 10.3719, Train Acc: 0.0906, Val Loss: 1.8594, Val Acc: 0.0000, LR: 0.000100
Epoch 3/100, Train Loss: 8.0491, Train Acc: 0.0897, Val Loss: 1.9389, Val Acc: 0.0000, LR: 0.000100
Epoch 4/100, Train Loss: 7.3580, Train Acc: 0.0966, Val Loss: 2.0323, Val Acc: 0.0000, LR: 0.000100
Epoch 5/100, Train Loss: 6.5509, Train Acc: 0.1009, Val Loss: 2.1067, Val Acc: 0.0000, LR: 0.000100
Epoch 6/100, Train Loss: 5.8787, Train Acc: 0.1051, Val Loss: 2.1430, Val Acc: 0.0000, LR: 0.000100
Epoch 7/100, Train Loss: 5.7531, Train Acc: 0.0983, Val Loss: 2.1252, Val Acc: 0.0000, LR: 0.000050
Epoch 8/100, Train Loss: 5.7737, Train Acc: 0.0974, Val Loss: 2.1135, Val Acc: 0.0000, LR: 0.000050
Epoch 9/100, Train Loss: 5.5885, Train Acc: 0.0863, Val Loss: 2.0953, Val Acc: 0.0000, LR: 0.000050
Epoch 10/100, Train Loss: 5.7513, Train Acc: 0.1060, Val Loss: 2.0923, Val Acc: 0.0000, LR: 0.0000

### Almost balanced but still overfitting


In [ ]:
import os
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import timm
from torch.optim import Adam
from PIL import Image
from collections import Counter
import numpy as np
from sklearn.metrics import f1_score

train_dir = '/content/drive/MyDrive/dataset_crossvit/extracted/train'
valid_dir = '/content/drive/MyDrive/dataset_crossvit/extracted/valid'

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    transforms.Lambda(lambda x: x + torch.randn_like(x) * 0.01)  # Corrected syntax
])

valid_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])


In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, gamma=1.0, weight=None, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.gamma = gamma  # Reduced gamma to lessen focus on hard examples
        self.weight = weight
        self.reduction = reduction

    def forward(self, input, target):
        ce_loss = F.cross_entropy(input, target, weight=self.weight, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = (1 - pt) ** self.gamma * ce_loss
        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        return focal_loss

class RoboflowCocoDataset(Dataset):
    def __init__(self, root_dir, annotation_file, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        with open(annotation_file, 'r') as f:
            self.coco_data = json.load(f)
        self.category_map = {cat['id']: idx for idx, cat in enumerate(self.coco_data['categories'])}
        self.class_names = [cat['name'] for cat in self.coco_data['categories']]
        self.image_info = {img['id']: img['file_name'] for img in self.coco_data['images']}
        self.annotations = {}
        for ann in self.coco_data['annotations']:
            img_id = ann['image_id']
            if img_id not in self.annotations:
                self.annotations[img_id] = []
            self.annotations[img_id].append(ann['category_id'])
        self.valid_image_ids = [img_id for img_id in self.image_info.keys() if img_id in self.annotations]
        print(f"Total images: {len(self.image_info)}, Images with annotations: {len(self.valid_image_ids)}")
        missing = set(self.image_info.keys()) - set(self.annotations.keys())
        if missing:
            print(f"Images missing annotations: {len(missing)}. Example IDs: {list(missing)[:5]}")

        labels = [self.annotations[img_id][0] for img_id in self.valid_image_ids]
        self.class_dist = Counter([self.class_names[self.category_map[l]] for l in labels])
        print("Class distribution:", self.class_dist)
        self.labels = [self.category_map[self.annotations[img_id][0]] for img_id in self.valid_image_ids]

    def __len__(self):
        return len(self.valid_image_ids)

    def __getitem__(self, idx):
        img_id = self.valid_image_ids[idx]
        img_path = os.path.join(self.root_dir, self.image_info[img_id])
        image = Image.open(img_path).convert('RGB')
        category_id = self.annotations[img_id][0]
        label = self.category_map[category_id]
        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:
# Load datasets
train_annotation_file = os.path.join(train_dir, '_annotations.coco.json')
valid_annotation_file = os.path.join(valid_dir, '_annotations.coco.json')

train_dataset = RoboflowCocoDataset(train_dir, train_annotation_file, transform=train_transform)
valid_dataset = RoboflowCocoDataset(valid_dir, valid_annotation_file, transform=valid_transform)

# Adjusted weighted sampling
class_counts = [train_dataset.class_dist[name] for name in train_dataset.class_names]
weights = 1.0 / torch.tensor(class_counts, dtype=torch.float)
# Soften the weights to reduce aggressive oversampling
weights = weights ** 0.5  # Square root to reduce the effect
sample_weights = torch.tensor([weights[label] for label in train_dataset.labels], dtype=torch.float)
sampler = WeightedRandomSampler(sample_weights, len(sample_weights))

train_loader = DataLoader(train_dataset, batch_size=32, sampler=sampler, num_workers=2)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False, num_workers=2)

print("Classes:", train_dataset.class_names)
print(f"Training samples: {len(train_dataset)}, Validation samples: {len(valid_dataset)}")
num_classes = len(train_dataset.class_names)

# Compute class weights for Focal Loss
total_samples = sum(class_counts)
weights = [total_samples / (num_classes * count) for count in class_counts]
weights = torch.tensor(weights, dtype=torch.float32)

# Add dropout to the model
model = timm.create_model('crossvit_15_240', pretrained=True, num_classes=num_classes, drop_rate=0.3)  # Add dropout
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print("Model loaded on:", device)

In [ ]:
def train_and_evaluate(model, train_loader, valid_loader, num_epochs=100, lr=0.0001):
    optimizer = Adam(model.parameters(), lr=lr, weight_decay=1e-3)  # Increased weight decay
    criterion = FocalLoss(gamma=1.0, weight=weights.to(device)).to(device)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5)

    # Early stopping parameters
    best_val_loss = float('inf')
    patience = 10
    patience_counter = 0

    # Freeze backbone initially
    for param in model.parameters():
        param.requires_grad = False
    for param in model.head.parameters():
        param.requires_grad = True

    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': [], 'val_f1': []}

    for epoch in range(num_epochs):
        if epoch == 10:
            for param in model.parameters():
                param.requires_grad = True
            print("Unfreezing all layers")

        model.train()
        train_loss, correct_train, total_train = 0.0, 0, 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            train_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct_train += (predicted == labels).sum().item()
            total_train += labels.size(0)
        train_loss /= len(train_loader.dataset)
        train_acc = correct_train / total_train

        model.eval()
        val_loss, correct_val, total_val = 0.0, 0, 0
        all_preds, all_labels = [], []
        with torch.no_grad():
            for inputs, labels in valid_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                correct_val += (predicted == labels).sum().item()
                total_val += labels.size(0)
                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
        val_loss /= len(valid_loader.dataset)
        val_acc = correct_val / total_val
        val_f1 = f1_score(all_labels, all_preds, average='weighted')  # Weighted F1-score for imbalance

        scheduler.step(val_loss)

        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        history['val_f1'].append(val_f1)

        print(f"Epoch {epoch+1}/{num_epochs}, "
              f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f}, "
              f"LR: {optimizer.param_groups[0]['lr']:.6f}")

        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            # Save the best model
            torch.save(model.state_dict(), 'best_model.pth')
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

    return history

In [ ]:
# Train and Evaluate
history = train_and_evaluate(model, train_loader, valid_loader, num_epochs=100)

Total images: 1176, Images with annotations: 1170
Images missing annotations: 6. Example IDs: [73, 747, 785, 537, 1019]
Class distribution: Counter({'Triple Bottom Reversal': 332, 'Ascending Triangle': 211, 'Symmetric Triangle': 177, 'Descending Triangle': 114, 'Head and Shoulder': 110, 'Double Top': 84, 'Inverse Head and Shoulder': 55, 'Cup and Handle': 40, 'Double Bottom': 23, 'flag-and-pole': 22, 'Falling Wedge': 2})
Total images: 289, Images with annotations: 288
Images missing annotations: 1. Example IDs: [52]
Class distribution: Counter({'Triple Bottom Reversal': 77, 'Ascending Triangle': 68, 'Symmetric Triangle': 35, 'Descending Triangle': 31, 'Double Top': 23, 'Head and Shoulder': 17, 'Inverse Head and Shoulder': 14, 'Double Bottom': 12, 'Cup and Handle': 6, 'flag-and-pole': 5})
Classes: ['flag-and-pole', 'Ascending Triangle', 'Cup and Handle', 'Descending Triangle', 'Double Bottom', 'Double Top', 'Falling Wedge', 'Head and Shoulder', 'Inverse Head and Shoulder', 'Symmetric Tri

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/110M [00:00<?, ?B/s]

Model loaded on: cuda
Epoch 1/100, Train Loss: 4.0880, Train Acc: 0.0744, Val Loss: 1.9283, Val Acc: 0.0382, Val F1: 0.0043, LR: 0.000100
Epoch 2/100, Train Loss: 4.3058, Train Acc: 0.0444, Val Loss: 1.9244, Val Acc: 0.0347, Val F1: 0.0040, LR: 0.000100
Epoch 3/100, Train Loss: 3.8988, Train Acc: 0.0496, Val Loss: 1.9131, Val Acc: 0.0417, Val F1: 0.0048, LR: 0.000100
Epoch 4/100, Train Loss: 4.4357, Train Acc: 0.0547, Val Loss: 1.9037, Val Acc: 0.0417, Val F1: 0.0049, LR: 0.000100
Epoch 5/100, Train Loss: 4.3833, Train Acc: 0.0607, Val Loss: 1.9060, Val Acc: 0.0347, Val F1: 0.0042, LR: 0.000100
Epoch 6/100, Train Loss: 4.1961, Train Acc: 0.0650, Val Loss: 1.9059, Val Acc: 0.0347, Val F1: 0.0041, LR: 0.000100
Epoch 7/100, Train Loss: 3.6558, Train Acc: 0.0564, Val Loss: 1.9047, Val Acc: 0.0312, Val F1: 0.0045, LR: 0.000100
Epoch 8/100, Train Loss: 4.3485, Train Acc: 0.0667, Val Loss: 1.9109, Val Acc: 0.0278, Val F1: 0.0047, LR: 0.000100
Epoch 9/100, Train Loss: 4.0366, Train Acc: 0.0513

In [ ]:
import os
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import timm
from torch.optim import Adam
from PIL import Image
from collections import Counter
import numpy as np
from sklearn.metrics import f1_score

train_dir = '/content/drive/MyDrive/dataset_crossvit/dataset_crossvit/extracted/train'
valid_dir = '/content/drive/MyDrive/dataset_crossvit/dataset_crossvit/extracted/valid'

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    transforms.Lambda(lambda x: x + torch.randn_like(x) * 0.01)
])

valid_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

class FocalLoss(nn.Module):
    def __init__(self, gamma=1.0, weight=None, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.weight = weight
        self.reduction = reduction

    def forward(self, input, target):
        ce_loss = F.cross_entropy(input, target, weight=self.weight, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = (1 - pt) ** self.gamma * ce_loss
        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        return focal_loss

class RoboflowCocoDataset(Dataset):
    def __init__(self, root_dir, annotation_file, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        with open(annotation_file, 'r') as f:
            self.coco_data = json.load(f)
        self.category_map = {cat['id']: idx for idx, cat in enumerate(self.coco_data['categories'])}
        self.class_names = [cat['name'] for cat in self.coco_data['categories']]
        self.image_info = {img['id']: img['file_name'] for img in self.coco_data['images']}
        self.annotations = {}
        for ann in self.coco_data['annotations']:
            img_id = ann['image_id']
            if img_id not in self.annotations:
                self.annotations[img_id] = []
            self.annotations[img_id].append(ann['category_id'])
        self.valid_image_ids = [img_id for img_id in self.image_info.keys() if img_id in self.annotations]
        print(f"Total images: {len(self.image_info)}, Images with annotations: {len(self.valid_image_ids)}")
        missing = set(self.image_info.keys()) - set(self.annotations.keys())
        if missing:
            print(f"Images missing annotations: {len(missing)}. Example IDs: {list(missing)[:5]}")

        labels = [self.annotations[img_id][0] for img_id in self.valid_image_ids]
        self.class_dist = Counter([self.class_names[self.category_map[l]] for l in labels])
        print("Class distribution:", self.class_dist)
        self.labels = [self.category_map[self.annotations[img_id][0]] for img_id in self.valid_image_ids]

    def __len__(self):
        return len(self.valid_image_ids)

    def __getitem__(self, idx):
        img_id = self.valid_image_ids[idx]
        img_path = os.path.join(self.root_dir, self.image_info[img_id])
        image = Image.open(img_path).convert('RGB')
        category_id = self.annotations[img_id][0]
        label = self.category_map[category_id]
        if self.transform:
            image = self.transform(image)
        return image, label

# Load datasets
train_annotation_file = os.path.join(train_dir, '_annotations.coco.json')
valid_annotation_file = os.path.join(valid_dir, '_annotations.coco.json')

train_dataset = RoboflowCocoDataset(train_dir, train_annotation_file, transform=train_transform)
valid_dataset = RoboflowCocoDataset(valid_dir, valid_annotation_file, transform=valid_transform)

# Adjusted weighted sampling
class_counts = [train_dataset.class_dist[name] for name in train_dataset.class_names]
weights = 1.0 / torch.tensor(class_counts, dtype=torch.float)
weights = weights ** 0.5  # Adjusted back to 0.5 for better balance
sample_weights = torch.tensor([weights[label] for label in train_dataset.labels], dtype=torch.float)
sampler = WeightedRandomSampler(sample_weights, len(sample_weights))

train_loader = DataLoader(train_dataset, batch_size=32, sampler=sampler, num_workers=4)  # Increased num_workers
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False, num_workers=4)

print("Classes:", train_dataset.class_names)
print(f"Training samples: {len(train_dataset)}, Validation samples: {len(valid_dataset)}")
num_classes = len(train_dataset.class_names)

# Compute class weights for Focal Loss
total_samples = sum(class_counts)
weights = [total_samples / (num_classes * count) for count in class_counts]
weights = torch.tensor(weights, dtype=torch.float32)

# Use a smaller CrossViT model to reduce training time
model = timm.create_model('crossvit_15_240', pretrained=True, num_classes=num_classes, drop_rate=0.3)  # Reduced dropout
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print("Model loaded on:", device)

def train_and_evaluate(model, train_loader, valid_loader, num_epochs=100, lr=0.0001):
    optimizer = Adam(model.parameters(), lr=lr, weight_decay=1e-3)  # Reduced weight decay
    criterion = FocalLoss(gamma=1.0, weight=weights.to(device)).to(device)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5)

    # Early stopping parameters
    best_val_loss = float('inf')
    patience = 10
    patience_counter = 0

    # Freeze backbone initially
    for param in model.parameters():
        param.requires_grad = False
    for param in model.head.parameters():
        param.requires_grad = True

    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': [], 'val_f1': []}

    for epoch in range(num_epochs):
        if epoch == 10:
            for param in model.parameters():
                param.requires_grad = True
            print("Unfreezing all layers")
            optimizer.param_groups[0]['lr'] = 0.00005  # Slightly higher learning rate

        model.train()
        train_loss, correct_train, total_train = 0.0, 0, 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            train_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct_train += (predicted == labels).sum().item()
            total_train += labels.size(0)
        train_loss /= len(train_loader.dataset)
        train_acc = correct_train / total_train

        model.eval()
        val_loss, correct_val, total_val = 0.0, 0, 0
        all_preds, all_labels = [], []
        with torch.no_grad():
            for inputs, labels in valid_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                correct_val += (predicted == labels).sum().item()
                total_val += labels.size(0)
                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
        val_loss /= len(valid_loader.dataset)
        val_acc = correct_val / total_val
        val_f1 = f1_score(all_labels, all_preds, average='weighted')
        # Per-class F1-scores
        per_class_f1 = f1_score(all_labels, all_preds, average=None)

        scheduler.step(val_loss)

        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        history['val_f1'].append(val_f1)

        print(f"Epoch {epoch+1}/{num_epochs}, "
              f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f}, "
              f"LR: {optimizer.param_groups[0]['lr']:.6f}")
        print("Per-class F1-scores:", {train_dataset.class_names[i]: f1 for i, f1 in enumerate(per_class_f1)})

        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), 'best_model.pth')
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

    return history

# Train and Evaluate
history = train_and_evaluate(model, train_loader, valid_loader, num_epochs=100)

Total images: 1176, Images with annotations: 1170
Images missing annotations: 6. Example IDs: [73, 747, 785, 537, 1019]
Class distribution: Counter({'Triple Bottom Reversal': 332, 'Ascending Triangle': 211, 'Symmetric Triangle': 177, 'Descending Triangle': 114, 'Head and Shoulder': 110, 'Double Top': 84, 'Inverse Head and Shoulder': 55, 'Cup and Handle': 40, 'Double Bottom': 23, 'flag-and-pole': 22, 'Falling Wedge': 2})
Total images: 289, Images with annotations: 288
Images missing annotations: 1. Example IDs: [52]
Class distribution: Counter({'Triple Bottom Reversal': 77, 'Ascending Triangle': 68, 'Symmetric Triangle': 35, 'Descending Triangle': 31, 'Double Top': 23, 'Head and Shoulder': 17, 'Inverse Head and Shoulder': 14, 'Double Bottom': 12, 'Cup and Handle': 6, 'flag-and-pole': 5})
Classes: ['flag-and-pole', 'Ascending Triangle', 'Cup and Handle', 'Descending Triangle', 'Double Bottom', 'Double Top', 'Falling Wedge', 'Head and Shoulder', 'Inverse Head and Shoulder', 'Symmetric Tri

### Still overfitting


In [ ]:
import os
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import timm
from torch.optim import Adam
from PIL import Image
from collections import Counter
import numpy as np
from sklearn.metrics import f1_score
from torch.cuda.amp import GradScaler, autocast  # For mixed precision training

train_dir = '/content/drive/MyDrive/dataset_crossvit/dataset_crossvit/extracted/train'
valid_dir = '/content/drive/MyDrive/dataset_crossvit/dataset_crossvit/extracted/valid'


In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    transforms.Lambda(lambda x: x + torch.randn_like(x) * 0.01)
])

valid_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, weight=None, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.gamma = gamma  # Increased gamma to focus on hard examples
        self.weight = weight
        self.reduction = reduction

    def forward(self, input, target):
        ce_loss = F.cross_entropy(input, target, weight=self.weight, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = (1 - pt) ** self.gamma * ce_loss
        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        return focal_loss

class RoboflowCocoDataset(Dataset):
    def __init__(self, root_dir, annotation_file, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        with open(annotation_file, 'r') as f:
            self.coco_data = json.load(f)
        self.category_map = {cat['id']: idx for idx, cat in enumerate(self.coco_data['categories'])}
        self.class_names = [cat['name'] for cat in self.coco_data['categories']]
        self.image_info = {img['id']: img['file_name'] for img in self.coco_data['images']}
        self.annotations = {}
        for ann in self.coco_data['annotations']:
            img_id = ann['image_id']
            if img_id not in self.annotations:
                self.annotations[img_id] = []
            self.annotations[img_id].append(ann['category_id'])
        self.valid_image_ids = [img_id for img_id in self.image_info.keys() if img_id in self.annotations]
        print(f"Total images: {len(self.image_info)}, Images with annotations: {len(self.valid_image_ids)}")
        missing = set(self.image_info.keys()) - set(self.annotations.keys())
        if missing:
            print(f"Images missing annotations: {len(missing)}. Example IDs: {list(missing)[:5]}")

        labels = [self.annotations[img_id][0] for img_id in self.valid_image_ids]
        self.class_dist = Counter([self.class_names[self.category_map[l]] for l in labels])
        print("Class distribution:", self.class_dist)
        self.labels = [self.category_map[self.annotations[img_id][0]] for img_id in self.valid_image_ids]

    def __len__(self):
        return len(self.valid_image_ids)

    def __getitem__(self, idx):
        img_id = self.valid_image_ids[idx]
        img_path = os.path.join(self.root_dir, self.image_info[img_id])
        image = Image.open(img_path).convert('RGB')
        category_id = self.annotations[img_id][0]
        label = self.category_map[category_id]
        if self.transform:
            image = self.transform(image)
        return image, label


In [ ]:
# Load datasets
train_annotation_file = os.path.join(train_dir, '_annotations.coco.json')
valid_annotation_file = os.path.join(valid_dir, '_annotations.coco.json')

train_dataset = RoboflowCocoDataset(train_dir, train_annotation_file, transform=train_transform)
valid_dataset = RoboflowCocoDataset(valid_dir, valid_annotation_file, transform=valid_transform)

# Adjusted weighted sampling
class_counts = [train_dataset.class_dist[name] for name in train_dataset.class_names]
weights = 1.0 / torch.tensor(class_counts, dtype=torch.float)
weights = weights ** 0.4  # Further softened weights
sample_weights = torch.tensor([weights[label] for label in train_dataset.labels], dtype=torch.float)
sampler = WeightedRandomSampler(sample_weights, len(sample_weights))

train_loader = DataLoader(train_dataset, batch_size=32, sampler=sampler, num_workers=2)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False, num_workers=2)

print("Classes:", train_dataset.class_names)
print(f"Training samples: {len(train_dataset)}, Validation samples: {len(valid_dataset)}")
num_classes = len(train_dataset.class_names)

# Compute class weights for Focal Loss
total_samples = sum(class_counts)
weights = [total_samples / (num_classes * count) for count in class_counts]
weights = torch.tensor(weights, dtype=torch.float32)

# Use crossvit_15_240 as it performs better
model = timm.create_model('crossvit_15_240', pretrained=True, num_classes=num_classes, drop_rate=0.4)  # Increased dropout
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print("Model loaded on:", device)


In [ ]:
def train_and_evaluate(model, train_loader, valid_loader, num_epochs=100, lr=0.0001):
    optimizer = Adam(model.parameters(), lr=lr, weight_decay=5e-3)  # Increased weight decay
    criterion = FocalLoss(gamma=2.0, weight=weights.to(device)).to(device)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5)
    scaler = GradScaler()  # For mixed precision training

    # Early stopping parameters
    best_val_loss = float('inf')
    patience = 10
    patience_counter = 0

    # Freeze backbone initially
    for param in model.parameters():
        param.requires_grad = False
    for param in model.head.parameters():
        param.requires_grad = True

    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': [], 'val_f1': []}

    for epoch in range(num_epochs):
        if epoch == 5:  # Reduced freezing period
            for param in model.parameters():
                param.requires_grad = True
            print("Unfreezing all layers")
            optimizer.param_groups[0]['lr'] = 0.00005

        model.train()
        train_loss, correct_train, total_train = 0.0, 0, 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            with autocast():  # Mixed precision training
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            scaler.step(optimizer)
            scaler.update()
            train_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct_train += (predicted == labels).sum().item()
            total_train += labels.size(0)
        train_loss /= len(train_loader.dataset)
        train_acc = correct_train / total_train

        model.eval()
        val_loss, correct_val, total_val = 0.0, 0, 0
        all_preds, all_labels = [], []
        with torch.no_grad():
            for inputs, labels in valid_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                with autocast():
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                correct_val += (predicted == labels).sum().item()
                total_val += labels.size(0)
                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
        val_loss /= len(valid_loader.dataset)
        val_acc = correct_val / total_val
        val_f1 = f1_score(all_labels, all_preds, average='weighted')
        per_class_f1 = f1_score(all_labels, all_preds, average=None)

        scheduler.step(val_loss)

        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        history['val_f1'].append(val_f1)

        print(f"Epoch {epoch+1}/{num_epochs}, "
              f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f}, "
              f"LR: {optimizer.param_groups[0]['lr']:.6f}")
        print("Per-class F1-scores:", {train_dataset.class_names[i]: f1 for i, f1 in enumerate(per_class_f1)})

        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), 'best_model.pth')
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

    return history


In [ ]:
# Train and Evaluate
history = train_and_evaluate(model, train_loader, valid_loader, num_epochs=100)

Total images: 1176, Images with annotations: 1170
Images missing annotations: 6. Example IDs: [73, 747, 785, 537, 1019]
Class distribution: Counter({'Triple Bottom Reversal': 332, 'Ascending Triangle': 211, 'Symmetric Triangle': 177, 'Descending Triangle': 114, 'Head and Shoulder': 110, 'Double Top': 84, 'Inverse Head and Shoulder': 55, 'Cup and Handle': 40, 'Double Bottom': 23, 'flag-and-pole': 22, 'Falling Wedge': 2})
Total images: 289, Images with annotations: 288
Images missing annotations: 1. Example IDs: [52]
Class distribution: Counter({'Triple Bottom Reversal': 77, 'Ascending Triangle': 68, 'Symmetric Triangle': 35, 'Descending Triangle': 31, 'Double Top': 23, 'Head and Shoulder': 17, 'Inverse Head and Shoulder': 14, 'Double Bottom': 12, 'Cup and Handle': 6, 'flag-and-pole': 5})
Classes: ['flag-and-pole', 'Ascending Triangle', 'Cup and Handle', 'Descending Triangle', 'Double Bottom', 'Double Top', 'Falling Wedge', 'Head and Shoulder', 'Inverse Head and Shoulder', 'Symmetric Tri

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/110M [00:00<?, ?B/s]

Model loaded on: cuda


<ipython-input-2-50a572049e65>:125: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()  # For mixed precision training
<ipython-input-2-50a572049e65>:152: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():  # Mixed precision training
<ipython-input-2-50a572049e65>:173: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


...

Epoch 21/100, Train Loss: 0.3488, Train Acc: 0.5291, Val Loss: 1.7893, Val Acc: 0.2465, Val F1: 0.2176, LR: 0.000025
Per-class F1-scores: {'flag-and-pole': np.float64(0.37209302325581395), 'Ascending Triangle': np.float64(0.0), 'Cup and Handle': np.float64(0.4666666666666667), 'Descending Triangle': np.float64(0.0), 'Double Bottom': np.float64(0.3611111111111111), 'Double Top': np.float64(0.38095238095238093), 'Falling Wedge': np.float64(0.1978021978021978), 'Head and Shoulder': np.float64(0.041666666666666664), 'Inverse Head and Shoulder': np.float64(0.05063291139240506), 'Symmetric Triangle': np.float64(0.0)}


<ipython-input-2-50a572049e65>:152: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():  # Mixed precision training
<ipython-input-2-50a572049e65>:173: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 22/100, Train Loss: 0.3072, Train Acc: 0.5496, Val Loss: 1.8302, Val Acc: 0.2014, Val F1: 0.1655, LR: 0.000025
Per-class F1-scores: {'flag-and-pole': np.float64(0.32727272727272727), 'Ascending Triangle': np.float64(0.0), 'Cup and Handle': np.float64(0.38461538461538464), 'Descending Triangle': np.float64(0.0), 'Double Bottom': np.float64(0.3541666666666667), 'Double Top': np.float64(0.09523809523809523), 'Falling Wedge': np.float64(0.1774193548387097), 'Head and Shoulder': np.float64(0.03508771929824561), 'Inverse Head and Shoulder': np.float64(0.0), 'Symmetric Triangle': np.float64(0.0)}


<ipython-input-2-50a572049e65>:152: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():  # Mixed precision training
<ipython-input-2-50a572049e65>:173: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 23/100, Train Loss: 0.2587, Train Acc: 0.5838, Val Loss: 1.8598, Val Acc: 0.2708, Val F1: 0.2201, LR: 0.000025
Per-class F1-scores: {'flag-and-pole': np.float64(0.4528301886792453), 'Ascending Triangle': np.float64(0.0), 'Cup and Handle': np.float64(0.3404255319148936), 'Descending Triangle': np.float64(0.0), 'Double Bottom': np.float64(0.37333333333333335), 'Double Top': np.float64(0.32727272727272727), 'Falling Wedge': np.float64(0.2191780821917808), 'Head and Shoulder': np.float64(0.08163265306122448), 'Inverse Head and Shoulder': np.float64(0.02564102564102564), 'Symmetric Triangle': np.float64(0.0)}
Early stopping at epoch 23
